In [ ]:

minimal.parser.add_argument("--delay_chunks", type=int, default=7, 
                           help="Delay in chunks for echo estimation")
minimal.parser.add_argument("--attenuation", type=float, default=0.5, 
                           help="Attenuation factor (0.0-0.99)")

Aquí defino las variables para cuando ejecute el comando por pantalla sepa que valores son cada uno con la definción de --delay_chunks y --attenuation. Por defecto le hemos puesto un valor medio de 0.5 de atenuación y 7 de chunks de retraso.

In [ ]:
class Feedback_Supression(buffer.Buffering):
    
    def __init__(self):
        super().__init__()
        logging.info(__doc__)
        self.played_chunk_history = []  # Lo que REPRODUCES (sale por altavoces)

        self.delay_in_chunks = minimal.args.delay_chunks
        self.attenuation = minimal.args.attenuation
        self.attenuation = min(max(self.attenuation, 0.0), 0.99)
 
        
        self.max_history = max(self.delay_in_chunks + 5, 30)
        
       

La clase feeback es la que va a reproducir los sonidos que capta el PC y al mismo tiempo la que va a eliminar el feedback de audio.  
Definimos las variables que vamos a usar:  
Delay de chuncks para eliminar los chunks que consideremos del audio
Atenuación para bajarle la intensidad a los chunks de feedback, asegurándose de que el valor está dentro de lo permitido 0.0-0.99
Played_chunck_history es eñ historial de chunks que se mantiene en memoria. Sirve para poder eliminar el eco, de ahí extrae los chunks los cuales atenua
Max_history pone como tope 30 chunks para asegurar un retardo max que eliminar


In [ ]:
 logging.info(f"AEC: delay={self.delay_in_chunks} chunks "
                    f"({self.delay_in_chunks * self.chunk_time * 1000:.1f} ms), "
                    f"attenuation={self.attenuation}")
    
    def _record_IO_and_play(self, ADC, DAC, frames, time, status):
        self.chunk_number = (self.chunk_number + 1) % self.CHUNK_NUMBERS


loggin.info muestra por pantalla los valores de las variables delay y atenuación usados. Convierte la duración de un chunk de audio de segundos a milisegundos, para que sea más fácil de interpretar.
chunk_number es el índice que permite que el buffer sea cíclico y aumenta en 1 cada vez que llega un chunk nuevo sin dejar que supere el límite de los chunks de memoria 30.

In [ ]:

       # Reproducir lo que llega desde red (o silencio)
        chunk = self.unbuffer_next_chunk()
        if chunk is None:
            chunk = self.zero_chunk
        self.play_chunk(DAC, chunk)

    

In [ ]:
Reproduce que el chunk que llega desde la red. Si no llega nada o el chunk está vacío reproduce silencio, si no el chunk que llega.  

In [ ]:

     

        # Guardar historial de salida
        self.played_chunk_history.append(chunk.copy())
        if len(self.played_chunk_history) > self.max_history:
            self.played_chunk_history.pop(0)

      

In [ ]:
Guarda una copia del chunk reproducido en el historial de chunks reproducidos. Si el historial supera el máximo permitido, elimina el chunk más antiguo.

In [ ]:
  # Estimar el eco según delay
        if len(self.played_chunk_history) >= self.delay_in_chunks:
            delayed_chunk = self.played_chunk_history[-self.delay_in_chunks]
        else:
            delayed_chunk = self.zero_chunk

        echo_estimation = self.attenuation * delayed_chunk

    

In [ ]:
Desde el historial escoge el chunk retrasado según el delay definido y lo atenúa según el valor definido.

In [ ]:
    # Cancelar el eco en la señal del micrófono
        filtered_ADC = ADC.astype(np.float32) - echo_estimation.astype(np.float32)
        filtered_ADC = np.clip(filtered_ADC, -32768, 32767).astype(np.int16)

        # Enviar la señal limpia
        packed_chunk = self.pack(self.chunk_number, filtered_ADC)
        self.send(packed_chunk)

        # 🔊 Reproducir también el micrófono procesado localmente (para verificar que se escucha)
        self.play_chunk(DAC, filtered_ADC)

Este es el bloque qye cancela el feedback:  
Para evitar desbordamientos en la resta normalizamos el valor de eco para que entre dentro de los 16bits con signo.  
astype convierte la señal a 16bits y clip garantiza que no se pase de los límetes de valores.  
Se empaqueta el número de chunk con el nuevo sonido limpio y se manda.  
Se reproduce el sonido en el altavoz.


In [ ]:
class Feedback_Supression__verbose(Feedback_Supression, buffer.Buffering__verbose):
    
    def __init__(self):
        super().__init__()
    
    def _record_IO_and_play(self, ADC, DAC, frames, time, status):
        if minimal.args.show_samples:
            self.show_recorded_chunk(ADC)
        
        super()._record_IO_and_play(ADC, DAC, frames, time, status)
        
        if minimal.args.show_samples:
            self.show_played_chunk(DAC)
        
        self.recorded_chunk = DAC
        self.played_chunk = ADC

In [ ]:
Este método muestra por pantalla los chunks grabados y reproducidos en el audio, mostrando los valores de las muestras.
Al mismo tiempo llama al método de la clase padre para que realice el procesamiento de audio.
Y guarda los chunks grabados y reproducidos en variables de la clase para poder acceder a ellos desde fuera.

CAMBIOS EN BUFFER.PY

In [ ]:
def unpack(self, packed_chunk):
        '''Splits the packed chunk into a chunk number and a chunk.'''
        (chunk_number,) = struct.unpack("!H", packed_chunk[:2])
        chunk = packed_chunk[2:]
        # Notice that struct.calcsize('H') = 2
        chunk = np.frombuffer(chunk, dtype=np.int16)
        chunk = chunk.reshape(minimal.args.frames_per_chunk, minimal.args.number_of_channels)
        return chunk_number, chunk


Hemos descomentado de la clase desempaquetar chunk = chunk.reshape(minimal.args.frames_per_chunk, minimal.args.number_of_channels), esto hace que podamos reproducir los chunks con la libreria sounddevice en el formato que necesita y no se escuche con distorsión.

In [ ]:
   if minimal.args.show_stats or minimal.args.show_samples or minimal.args.show_spectrum:
        intercom = Buffering__verbose(minimal.args)
        intercom = Buffering__verbose()
    else:
        intercom = Buffering()

Hemos descomentado la línea de  intercom = Buffering__verbose(minimal.args) para poder pasarle a la clase minimal los argumentos de delay y attenuation